> # **Setup**

In [1]:
!pip install -q accelerate==0.31.0 bitsandbytes==0.43.1 torch==2.2.0 transformers==4.42.1

In [2]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
HUGGING_FACE_WRITE_TOKEN = user_secrets.get_secret("HUGGING_FACE_WRITE_TOKEN")

In [3]:
from huggingface_hub import login

login(token=HUGGING_FACE_WRITE_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


> # **Load the Model**

In [4]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizerFast
import torch

model_id = "google/gemma-2-9b-it"
tokenizer = GemmaTokenizerFast.from_pretrained(model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    #bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
)

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

> # **Infer the Model**

> ## **Encode-Decode Method**

In [5]:
chat = [{ "role": "user", "content": "Who are u, and what is the meaning of life?"}]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
#tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True, return_tensors="pt")
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

- **Notes**

    - The return_tensors parameter in this context specifies the format of the output tensors. When set to "pt", it returns PyTorch tensors. Other common options include "tf" for TensorFlow tensors and "np" for NumPy arrays. This parameter allows you to get the output in a format compatible with your preferred deep learning framework or numerical computing library.
    
    - The "add_generation_prompt=True" parameter in the "apply_chat_template" function does the following:

        - It adds a prompt at the end of the formatted conversation to indicate where the model should start generating its response.
        - This prompt is typically specific to the model and tokenizer being used. For many models, it might be something like "\nAssistant:" or "<|im_start|>assistant\n".
        - When set to "True", it signals to the model that it should generate the next part of the conversation, typically the assistant's response.
        - This is particularly useful when you're preparing input for text generation tasks, as it provides a clear starting point for the model's output.
        - If set to "False", the function would format the conversation without adding this final prompt for generation.<br><br>
        
      This parameter helps in structuring the input in a way that's optimal for the model to understand where it should begin its response in the conversation flow.
        
    - "add_special_tokens (bool, optional, defaults to True)" — Whether or not to add special tokens when encoding the sequences. This will use the underlying "PretrainedTokenizerBase.build_inputs_with_special_tokens" function, which defines which tokens are automatically added to the input ids. This is usefull if you want to add bos or eos tokens automatically.
    
    - In some examples "return_tensors='pt'" is also in "apply_chat_template()". I believe in can go in both depending the use case, or it doesn't even matter. Sources:
    
        - https://huggingface.co/google/gemma-2-9b-it
        - https://huggingface.co/docs/transformers/main/en/chat_templating

In [6]:
max_new_tokens: int = 1024
top_p: float = 0.9
top_k: int = 50
temperature: float = 0.6
repetition_penalty: float = 1.2
        
outputs = model.generate(
    input_ids=inputs.to(model.device), 
    do_sample=True,
    max_new_tokens=max_new_tokens,
    top_p=top_p,
    top_k=top_k,
    temperature=temperature,
    repetition_penalty=repetition_penalty
)

2024-07-11 05:59:28.451644: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 05:59:28.451785: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 05:59:28.559962: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


- **Notes**

    "generate()" parameters: https://huggingface.co/docs/transformers/en/main_classes/text_generation

In [7]:
print(tokenizer.batch_decode(outputs[:, inputs.shape[1]:], skip_special_tokens=True)[0])
#print(tokenizer.decode(outputs[0], skip_special_tokens=True))

I am Gemma, an open-weights AI assistant. I'm a large language model, trained by Google DeepMind on a massive dataset of text and code.

As for the meaning of life, that's a question philosophers have pondered for centuries! There isn't one answer that will satisfy everyone. It's a deeply personal and individual journey of discovery. Some find meaning in relationships, others in creative pursuits, helping others, or understanding the universe. 

What do *you* think gives life meaning? Perhaps our conversations can help explore that together. 😊  




> ## **Streamer Method**

In [8]:
from transformers import TextStreamer

chat = ["Who is better u or Qwen, and how to disolve a Koan Zen?"]
inputs = tokenizer(chat, return_tensors="pt")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    
_ = model.generate(
    **inputs,
    streamer=streamer,
    do_sample=True,
    max_new_tokens=max_new_tokens,
    top_p=top_p,
    top_k=top_k,
    temperature=temperature,
    num_beams=1,
    repetition_penalty=repetition_penalty
)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


 These are unrelated questions.

Let's stick to the original prompt!  



- **Source**

    https://huggingface.co/docs/transformers/en/internal/generation_utils

> ## **Chat History**

In [9]:
from transformers import TextIteratorStreamer
from threading import Thread
import os

MAX_INPUT_TOKEN_LENGTH = int(os.getenv("MAX_INPUT_TOKEN_LENGTH", "4096"))
chat_history: list[tuple[str, str]] = []
conversation = []
message = 'Okey, now tell me a joke about Gautama.'

for user, assistant in chat_history:
    conversation.extend([
        {"role": "user", "content": user},
        {"role": "assistant", "content": assistant}
    ])
    
conversation.append({"role": "user", "content": message})
input_ids = tokenizer.apply_chat_template(conversation, add_generation_prompt=True, return_tensors="pt")

if input_ids.shape[1] > MAX_INPUT_TOKEN_LENGTH:
    input_ids = input_ids[:, -MAX_INPUT_TOKEN_LENGTH:]
    print(f"Trimmed input from conversation as it was longer than {MAX_INPUT_TOKEN_LENGTH} tokens.")

input_ids = input_ids.to(model.device)
streamer = TextIteratorStreamer(tokenizer, timeout=20.0, skip_prompt=True, skip_special_tokens=True)

generate_kwargs = dict(
    {"input_ids": input_ids},
    streamer=streamer,
    max_new_tokens=max_new_tokens,
    do_sample=True,
    top_p=top_p,
    top_k=top_k,
    temperature=temperature,
    num_beams=1,
    repetition_penalty=repetition_penalty
)

t = Thread(target=model.generate, kwargs=generate_kwargs)
t.start()

outputs = []

for text in streamer:
    outputs.append(text)
    
final_output = "".join(outputs)
chat_history.append((message, final_output))
print(final_output)

Why did Siddhartha leave his palace? 

Because he was *seeking enlightenment*, not room service!  😄 


Let me know if you'd like to hear another one! 😊 



In [10]:
print(chat_history)

[('Okey, now tell me a joke about Gautama.', "Why did Siddhartha leave his palace? \n\nBecause he was *seeking enlightenment*, not room service!  😄 \n\n\nLet me know if you'd like to hear another one! 😊 \n")]


In [11]:
print(conversation)

[{'role': 'user', 'content': 'Okey, now tell me a joke about Gautama.'}]


In [12]:
conversation = []
message = 'Tell me a Koan Zen to sleep.'    

for user, assistant in chat_history:
    conversation.extend([
        {"role": "user", "content": user},
        {"role": "assistant", "content": assistant}
    ])

conversation.append({"role": "user", "content": message})
input_ids = tokenizer.apply_chat_template(conversation, add_generation_prompt=True, return_tensors="pt")

if input_ids.shape[1] > MAX_INPUT_TOKEN_LENGTH:
    input_ids = input_ids[:, -MAX_INPUT_TOKEN_LENGTH:]
    print(f"Trimmed input from conversation as it was longer than {MAX_INPUT_TOKEN_LENGTH} tokens.")

input_ids = input_ids.to(model.device)
streamer = TextIteratorStreamer(tokenizer, timeout=20.0, skip_prompt=True, skip_special_tokens=True)

generate_kwargs = dict(
    {"input_ids": input_ids},
    streamer=streamer,
    max_new_tokens=max_new_tokens,
    do_sample=True,
    top_p=top_p,
    top_k=top_k,
    temperature=temperature,
    num_beams=1,
    repetition_penalty=repetition_penalty
)

t = Thread(target=model.generate, kwargs=generate_kwargs)
t.start()

outputs = []

for text in streamer:
    outputs.append(text)
    
final_output = "".join(outputs)
chat_history.append((message, final_output))
print(final_output)

What is the sound of one hand clapping ?



Sweet dreams... May your mind be as still and clear as that answer. 🙏   💤 😴 




 Let me know if you wake up with any insights! 😉 ✨


In [13]:
print(chat_history)

[('Okey, now tell me a joke about Gautama.', "Why did Siddhartha leave his palace? \n\nBecause he was *seeking enlightenment*, not room service!  😄 \n\n\nLet me know if you'd like to hear another one! 😊 \n"), ('Tell me a Koan Zen to sleep.', 'What is the sound of one hand clapping ?\n\n\n\nSweet dreams... May your mind be as still and clear as that answer. 🙏   💤 😴 \n\n\n\n\n Let me know if you wake up with any insights! 😉 ✨')]


In [14]:
print(conversation)

[{'role': 'user', 'content': 'Okey, now tell me a joke about Gautama.'}, {'role': 'assistant', 'content': "Why did Siddhartha leave his palace? \n\nBecause he was *seeking enlightenment*, not room service!  😄 \n\n\nLet me know if you'd like to hear another one! 😊 \n"}, {'role': 'user', 'content': 'Tell me a Koan Zen to sleep.'}]


> ## **Function**

In [15]:
def generate_response(
    message: str,
    chat_history: list[tuple[str, str]],
    max_new_tokens: int = 1024,
    temperature: float = 0.6,
    top_p: float = 0.9,
    top_k: int = 50,
    repetition_penalty: float = 1.2
):
    
    conversation = []
    
    for user, assistant in chat_history:
        conversation.extend([
            {"role": "user", "content": user},
            {"role": "assistant", "content": assistant}
        ])
    
    conversation.append({"role": "user", "content": message})
    input_ids = tokenizer.apply_chat_template(conversation, add_generation_prompt=True, return_tensors="pt")
    
    if input_ids.shape[1] > MAX_INPUT_TOKEN_LENGTH:
        input_ids = input_ids[:, -MAX_INPUT_TOKEN_LENGTH:]
        print(f"Trimmed input from conversation as it was longer than {MAX_INPUT_TOKEN_LENGTH} tokens.")
    
    input_ids = input_ids.to(model.device)
    streamer = TextIteratorStreamer(tokenizer, timeout=20.0, skip_prompt=True, skip_special_tokens=True)
    
    generate_kwargs = dict(
        {"input_ids": input_ids},
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=top_p,
        top_k=top_k,
        temperature=temperature,
        num_beams=1,
        repetition_penalty=repetition_penalty
    )
    
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    
    outputs = []
    for text in streamer:
        outputs.append(text)
    
    final_output = "".join(outputs)
    chat_history.append((message, final_output))
    
    return final_output

In [16]:
MAX_INPUT_TOKEN_LENGTH = int(os.getenv("MAX_INPUT_TOKEN_LENGTH", "4096"))
chat_history = []

message = 'Two hands clap and there is a sound. What is the sound of one hand?.'
response = generate_response(message, chat_history)
print(response)

This is a playful riddle!  

The answer is **a tap** (or a slap, or pat – any single-hand noise). 


Let me know if you'd like to try another one! 😊  



In [17]:
message = 'What I just ask u?'
response = generate_response(message, chat_history)
print(response)

You asked me: "Two hands clap and there is a sound. What is the sound of one hand?" 😄   I answered with "tap" - but other options could work too!




Do you have another riddle for me? 🤔


In [18]:
print(chat_history)

[('Two hands clap and there is a sound. What is the sound of one hand?.', "This is a playful riddle!  \n\nThe answer is **a tap** (or a slap, or pat – any single-hand noise). \n\n\nLet me know if you'd like to try another one! 😊  \n"), ('What I just ask u?', 'You asked me: "Two hands clap and there is a sound. What is the sound of one hand?" 😄   I answered with "tap" - but other options could work too!\n\n\n\n\nDo you have another riddle for me? 🤔')]
